# EFASv5 - extraer caudal
***

***Autor:** Jesús Casado*<br>
***Fecha:** 08-08-2023*<br>

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
# from shapely.geometry import Point
from tqdm.notebook import tqdm

## Configuración

In [ ]:
path_EFAS = '../../EFASv5/iberia/'

path_out = '../EFASv5/discharge/reanalysis/'
if os.path.exists(path_out) is False:
    os.makedirs(path_out)
    
estaciones = 'EFAS' # ROEA

folder = {'EFAS': 'reporting_poitns',
          'ROEA': 'selected_stations'}

## Estaciones
### Estaciones ROEA

In [ ]:
if estaciones == 'ROEA':
    
    # cargar shapefile de puntos
    puntos = gpd.read_file('../estaciones_seleccion_EFAS.shp')
    puntos.set_index('indroea', drop=True, inplace=True)

    # xarrays con las coordenadas de las estaciones
    x = xr.DataArray(puntos.geometry.x, dims='id')
    y = xr.DataArray(puntos.geometry.y, dims='id')

### Puntos de EFAS

In [ ]:
if estaciones == 'EFAS':
    
    # cargar shapefile de puntos
    puntos = gpd.read_file(f'{path_EFAS}reporting_points.shp')
    puntos.geometry = gpd.points_from_xy(puntos.LisfloodX, puntos.LisfloodY)
    puntos.set_index('ObsID', drop=True, inplace=True)

    # xarrays con las coordenadas de las estaciones
    x = xr.DataArray(puntos.geometry.x, dims='id')
    y = xr.DataArray(puntos.geometry.y, dims='id')

## Caudal
### Reanálisis EFAS

In [ ]:
# extreaer series de caudal archivo a archivo
files = glob.glob(f'{path_EFAS}reanalysis/dis_*.nc')
for file in tqdm(files):
    
    # check if output file already exists
    file_out = '{0}{1}/{2}'.format(path_out, folder[estaciones], file.split('\\')[-1])
    if os.path.exists(file_out):
        continue
    
    # load data
    dis = xr.open_dataset(file, chunks='auto')['dis']
    dis.close()
    
    # extract discharge for the selected stations
    dis = dis.sel(lon=x, lat=y, method='nearest')
    dis = dis.drop(['lon', 'lat'])
    
    # resample to daily timesteps. WARNING! The data is shifted 6 h!!!
    #dis['time'] = dis.time + np.timedelta64(6, 'h')
    dis = dis.resample(time='1D').mean()
    
    break
    
    # compute the lazy DataArray
    dis = dis.compute()

    # export
    dis.name = 'discharge'
    dis.to_netcdf(file_out)